# Year-round North Atlantic-European Weather Regimes

Following the example in Part 4 of [github.com/cmgrams/wr_data_package_era5](https://github.com/cmgrams/wr_data_package_era5/blob/main/wr_data_package_V1.0/scripts_first_steps/WR_read_example.ipynb), compute the regime index for the seven-regime classification of [Grams (2026, in review)](https://doi.org/10.5194/egusphere-2025-6385).

In [1]:
import pooch
import numpy as np
import pandas as pd
import xarray as xr

from earthkit.meteo import regimes

## Get the regime classification data

Retrieve data from https://zenodo.org/records/18154492.

In [2]:
files = pooch.retrieve(
    url="doi:10.5281/zenodo.18154492/wr_data_package_V1.1.zip",
    known_hash="dc942ff2a1b3da6dedd3b0b2fadda017fff9e8fc10228ace31c2209a9be7dc62",
    processor=pooch.Unzip()
)

def get_file(name):
    for file in files:
        if file.endswith(name):
            return file
    raise FileNotFoundError(name)

## Regime pattern normalisation based on day-of-year

Load the file with normalisation weights from the repository.

In [3]:
mod_ds = xr.open_dataset(get_file("wr_data/EOFs_WRs.nc"))

/opt/homebrew/Caskroom/miniforge/base/envs/earthkit-meteo/lib/python3.14/site-packages/xarray/backends/plugins.py:109: RuntimeWarning: Engine 'earthkit' loading failed:
cannot import name 'array_to_numpy' from 'earthkit.utils.array' (/opt/homebrew/Caskroom/miniforge/base/envs/earthkit-meteo/lib/python3.14/site-packages/earthkit/utils/array/__init__.py)
  external_backend_entrypoints = backends_dict_from_pkg(entrypoints_unique)


Create a lookup table for the normalisation weights and define a function that finds the appropriate weight for a given date.

In [4]:
mod = 1. / mod_ds["normwgt"].to_series()
mod.index = pd.Index(zip(mod.index.month, mod.index.day, mod.index.hour))

def pattern_normalisation_weight(date):
    date = np.asarray(date)
    shp = date.shape
    date = pd.to_datetime(date.flatten())
    if isinstance(date, pd.Timestamp):
        return mod.loc[(date.month, date.day, date.hour)]
    else:
        idx = list(zip(date.month, date.day, date.hour))
        return mod.loc[idx].values.reshape(shp)

## Load the regime patterns

Patterns are stored in a NetCDF file in the repository.

In [5]:
pattern_ds = xr.open_dataset(get_file("wr_data/Normed_Z0500-patterns_EOFdomain.nc"))

Seven base patterns. The normalisation of projections is implemented via a modulation of the regime patterns, i.e., we vary the amplitude of the patterns.

In [6]:
patterns = regimes.ModulatedPatterns(
    labels=pattern_ds.attrs["ClassNames"].split(),
    grid={
        "grid": [0.5, 0.5],
        "area": [90, -80, 30, 40]  # 30-90°N, 80°W-40°E
    },
    patterns=pattern_ds["Z0500_mean"].values,
    modulator=pattern_normalisation_weight
)

**Note:** the regime data from this repository uses ascending order for the latitude coordinate. The latitude ordering of fields projected onto these patterns must match.

## Project the test field onto the patterns

Load the test field provided with the dataset. This field contains pre-processed Z500 anomalies ready for projection.

In [7]:
ds_field = xr.open_dataset(get_file("wr_data_package_V1.1/example_data/Z0500_20250601_00.nc")).squeeze()

# Extract values for the EUR-ATL region and create associated coordinates
# following the example in the reference notebook
field = ds_field["Z0"].values[240:361,200:441]

Create area-based weights for the grid points to use in the projection.

In [8]:
lat = np.linspace(ds_field.attrs["domymin"], ds_field.attrs["domymax"], ds_field["Z0"].shape[0])

weights = np.cos(np.deg2rad(lat))
weights = np.repeat(weights[240:361], field.shape[1]).reshape(field.shape)

Project onto the regime patterns, supply the valid date of the field for the modulator function to select the normalisation weight.

In [9]:
projections = regimes.array.project(field, patterns, weights, date=np.datetime64("2025-06-01 00:00"))

## Compute the regime index

Standardise the projections to obtain the regime index.
Reference values are given in the notebook:

    weather regime indices for  20250601_00
    0.8218320408050166 AT
    1.1691867614026132 ZO
    1.0489664646400954 ScTr
    -0.6948883613466961 AR
    -0.5738999234610754 EuBL
    -0.9323144170607468 ScBL
    -0.6369122243737739 GL

Read the standardisation parameters (mean and standard deviation) from the text file in the repo.

In [10]:
def read_wri_std_parameters(path):
    f.readline()
    name = f.readline().strip().split()
    mean = f.readline().strip().split()[1:]
    std = f.readline().strip().split()[1:]
    return (
        {n: float(v) for n, v in zip(name, mean)},
        {n: float(v) for n, v in zip(name, std)}
    )
    
with open(get_file("wr_data/WRI_std_params.txt"), "r") as f:
    norm_mean, norm_std = read_wri_std_parameters(f)

Compute the regime index.

In [11]:
regimes.array.regime_index(projections, norm_mean, norm_std)

{'AT': np.float32(0.8218321),
 'ZO': np.float32(1.169187),
 'ScTr': np.float32(1.0489665),
 'AR': np.float32(-0.6948884),
 'EuBL': np.float32(-0.5738999),
 'ScBL': np.float32(-0.9323146),
 'GL': np.float32(-0.6369122)}